In [7]:
%%writefile requirements.txt
torch==2.7.1
transformers==4.44.2
scikit-learn==1.6.1
pandas==2.3.0
numpy==2.2.6

Writing requirements.txt


In [2]:
# !pip install -r requirements.txt


In [1]:
!mkdir app inputs outputs

In [2]:
%%writefile app/bot.py
import pickle
import socket
import atexit

# Known bot suffixes used for reverse DNS validation
KNOWN_BOTS = {
    "Googlebot": [".googlebot.com"],
    "Bingbot": [".search.msn.com"],
    "AhrefsBot": [".ahrefs.com", ".ahrefs.net"],
    "YandexBot": [".yandex.ru", ".yandex.com", ".yandex.net"],
    "SemrushBot": [".semrush.com"],
    "DuckDuckBot": [".duckduckgo.com"],
    "MJ12bot": [".majestic12.co.uk"],
    "Slurp": [".crawl.yahoo.net"],
    "Applebot": [".apple.com"]
}

# Cache file paths for verified and spoofed bot IPs
VERIFIED_IP_FILE = "verified_bots.pkl"
SPOOFED_IP_FILE = "spoofed_bots.pkl"

def pickle_load(path):
    """
    Load a Python set object from a pickle file.
    """
    try:
        with open(path, "rb") as f:
            return pickle.load(f)
    except Exception:
        return set()

# Load IP caches early to avoid NameError during runtime
verified_bot_ips = pickle_load(VERIFIED_IP_FILE)
spoofed_bot_ips = pickle_load(SPOOFED_IP_FILE)

def save_ip_caches():
    """
    Persist the sets of verified and spoofed bot IP addresses to disk.
    """
    with open(VERIFIED_IP_FILE, "wb") as f:
        pickle.dump(verified_bot_ips, f)
    with open(SPOOFED_IP_FILE, "wb") as f:
        pickle.dump(spoofed_bot_ips, f)

atexit.register(save_ip_caches)

def reverse_dns(ip):
    """
    Perform a reverse DNS lookup for a given IP address.
    """
    try:
        return socket.gethostbyaddr(ip)[0]
    except socket.herror:
        return None

def forward_dns(hostname):
    """
    Perform a forward DNS lookup for a given hostname.
    """
    try:
        return socket.gethostbyname(hostname)
    except socket.gaierror:
        return None

def is_valid_bot(ip, ua):
    """
    Determine if a request is from a legitimate search engine bot.
    """
    if ip in verified_bot_ips:
        return True
    if ip in spoofed_bot_ips:
        return False

    for bot, suffixes in KNOWN_BOTS.items():
        if bot.lower() in ua.lower():
            rdns = reverse_dns(ip)
            if not rdns or not any(rdns.endswith(sfx) for sfx in suffixes):
                spoofed_bot_ips.add(ip)
                return False
            if forward_dns(rdns) != ip:
                spoofed_bot_ips.add(ip)
                return False
            verified_bot_ips.add(ip)
            return True

    return False  # Not a known bot

Writing app/bot.py


In [4]:
%%writefile app/decoder.py
import os
import re
import sys
import codecs
import csv  # <<< Added for streaming
import argparse
import urllib.parse
import base64
# import pandas as pd # <<< Removed pandas dependency for streaming CSV
from app.bot import is_valid_bot

# Variables
log_pattern = re.compile(
    r'(?P<ip>\S+) - - \[(?P<time>[^\]]+)\] '
    r'"(?P<method>\S+) (?P<url>\S+) (?P<protocol>[^"]+)" '
    r'(?P<status>\d+) (?P<size>\d+) '
    r'"(?P<referrer>[^"]*)" "(?P<user_agent>[^"]*)" "(?P<extra>[^"]*)"'
)

# Functions
def esc_nl(text):
    """
    Escape newline and carriage return characters in a string.
    """
    return text.replace('\n', '\\n').replace('\r', '\\r').strip()


def dec_url(text):
    """
    Decode a URL-encoded string up to two iterations.
    """
    try:
        first = urllib.parse.unquote(text)
        if first == text:
            return text

        second = urllib.parse.unquote(first)
        if second == first:
            return first

        return second
    except Exception:
        return text


def dec_esc(text):
    """
    Decode escaped character sequences such as \\xNN and \\uNNNN.
    """
    try:
        if '\\x' in text or '\\u' in text:
            decoded = codecs.escape_decode(text.encode())[0].decode('utf-8', errors='replace')
            return decoded
        return text
    except Exception:
        return text


def dec_base64(text):
    """
    Detect and decode a Base64-encoded segment in the final URL path.
    """
    try:
        last_part = text.rsplit("/", 1)[-1]

        # Heuristic: reasonably long, valid base64 characters
        if re.fullmatch(r'[A-Za-z0-9+/=]{8,}', last_part):
            decoded = base64.b64decode(last_part, validate=False).decode('utf-8', errors='ignore')
            annotated = f"{text}(base64:{decoded})"
            return annotated

        return text
    except Exception:
        return text


def dec_combined(text):
    """
    Apply a sequence of decoding techniques to a string:
    1. URL decoding (up to two iterations)
    2. Escape sequence decoding (e.g., \\xNN, \\uNNNN)
    3. Base64 decoding on the last URL path segment
    """
    text = dec_url(text)
    text = dec_esc(text)
    text = dec_base64(text)
    return text


def parse_dec_line(line):
    """
    Parse and decode a single NGINX access log line.
    """
    match = log_pattern.match(line)
    if not match:
        return None, None  # Unparsable log line

    fields = match.groupdict()

    # Decode URL field (multi-step decoding)
    fields['url'] = dec_combined(fields['url'])

    # Decode referrer field (only take the decoded text, not flags)
    fields['referrer'] = dec_combined(fields['referrer'])

    # Apply newline escaping cleanup
    for key in fields:
        fields[key] = esc_nl(fields[key])

    decoded = (
        f'{fields["ip"]} - - [{fields["time"]}] '
        f'"{fields["method"]} {fields["url"]} {fields["protocol"]}" '
        f'{fields["status"]} {fields["size"]} '
        f'"{fields["referrer"]}" "{fields["user_agent"]}" "{fields["extra"]}"'
    )

    return decoded, fields


def parse_dec_file(in_path, out_path):
    """
    Decode and clean all entries in a log file and write the results to a new file.
    """
    with open(in_path, 'r', encoding='utf-8', errors='replace') as in_file, \
        open(out_path, 'w', encoding='utf-8') as out_file:

        for line in in_file:
            decoded, fields = parse_dec_line(line)

            if not fields:
                continue  # Skip unparsed line

            if is_valid_bot(fields['ip'], fields['user_agent']):
                continue  # Skip valid bot

            out_file.write(f"{decoded}\n")


# Hapus parse_dec_file_to_dataframe dan parse_dec_file_to_csv (lama)
# Ganti dengan fungsi streaming yang baru
def parse_dec_file_stream_to_csv(in_path, out_path):
    """
    Decode and clean all entries in a log file and stream them to a CSV file.
    This avoids loading all data into RAM (MemoryError).
    """
    print("⏳ Starting streaming decode to CSV...")
    
    with open(in_path, 'r', encoding='utf-8', errors='replace') as in_file, \
         open(out_path, 'w', encoding='utf-8', newline='') as out_file:

        writer = None
        
        for no, line in enumerate(in_file, 1):
            _, fields = parse_dec_line(line)

            if not fields:
                continue  # Skip unparsed line

            if is_valid_bot(fields['ip'], fields['user_agent']):
                continue  # Skip valid bot

            # Add line number (optional, but in the original code)
            fields['no'] = no
            
            # Initialize CSV writer after the first record is processed (to get fieldnames)
            if writer is None:
                fieldnames = list(fields.keys())
                writer = csv.DictWriter(out_file, fieldnames=fieldnames)
                writer.writeheader()
                
            writer.writerow(fields)
            
            # Progress update (optional)
            if no % 100000 == 0:
                print(f"  Processed {no:,} lines...")

    print("✅ Streaming decode complete.")


if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="NGINX log decoder.")
    parser.add_argument("in_file", help="NGINX log file")
    parser.add_argument("out_file", help="The decoded NGINX log file")
    parser.add_argument("--csv", action="store_true", help="Save the output in CSV format.")

    # Parse the arguments
    args = parser.parse_args()

    if not os.path.exists(args.in_file):
        print(f"❌ File not found: '{args.in_file}'")
        sys.exit(1)

    out_dir = os.path.dirname(args.out_file)
    if out_dir and not os.path.exists(out_dir):
        # NOTE: Instead of checking existence, better to create it if it doesn't exist
        try:
            os.makedirs(out_dir, exist_ok=True)
        except OSError as e:
            print(f"❌ Cannot create output directory: '{out_dir}' - {e}")
            sys.exit(1)
        
    if args.csv:
        # Use the memory-efficient streaming function
        parse_dec_file_stream_to_csv(args.in_file, args.out_file)
    else:
        parse_dec_file(args.in_file, args.out_file)

    print(f"✅ Log file successfully converted to {args.out_file}")

Overwriting app/decoder.py


In [12]:
%%writefile app/main.py
# app/main.py
import os
import re
import csv
import sys
import argparse
import warnings
import time 
from collections import Counter
from urllib.parse import urlparse, unquote

import numpy as np
import pandas as pd
import torch
from transformers import BertTokenizerFast, BertModel
from sklearn.preprocessing import normalize
from sklearn.metrics import pairwise_distances, silhouette_score, davies_bouldin_score

# =========================
# IMPORT KHUSUS GPU (CUML/CUPY) DAN FALLBACK KE SKLEARN (CPU)
# =========================
try:
    # Coba import CuML dan CuPy
    from cuml.cluster import OPTICS as CumlOPTICS
    import cupy as cp
    
    # Periksa apakah CuPy benar-benar bisa menggunakan GPU
    if cp.cuda.is_available():
        HAS_GPU_CLUSTERING = True
        OPTICS_IMPL = CumlOPTICS
    else:
        # Paksakan fallback jika CUDA tidak tersedia di CuPy
        raise RuntimeError("CuPy terinstal tetapi GPU tidak terdeteksi atau siap.")

# Tangkap ImportError, RuntimeError, atau Exception lain
except (ImportError, RuntimeError, Exception) as e: 
    # Fallback ke Scikit-learn OPTICS
    #print(f"DEBUG: CuML/CuPy import atau inisialisasi gagal: {e}") # Buka jika perlu debugging
    from sklearn.cluster import OPTICS as SklearnOPTICS
    OPTICS_IMPL = SklearnOPTICS
    cp = np 
    HAS_GPU_CLUSTERING = False


# =========================
# Konfigurasi & Inisialisasi
# =========================

# Sembunyikan FutureWarning transformers yang tidak fatal
warnings.filterwarnings(
    "ignore",
    message="`clean_up_tokenization_spaces` was not set",
    category=FutureWarning,
)

# Pilih device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if HAS_GPU_CLUSTERING and device.type == "cuda":
    print("✅ GPU Acceleration aktif untuk BERT dan OPTICS (via CuML).")
elif HAS_GPU_CLUSTERING and device.type == "cpu":
    print("⚠️ Peringatan: CuML terinstal, tetapi PyTorch menggunakan CPU. Klastering GPU tidak efektif.")
else:
    print("❌ GPU Clustering (CuML) tidak terinstal. Menggunakan Scikit-learn (CPU).")


# Muat tokenizer & model BERT
TOKENIZER = BertTokenizerFast.from_pretrained("bert-base-uncased")
torch_dtype = torch.float16 if device.type == "cuda" else torch.float32
MODEL = BertModel.from_pretrained("bert-base-uncased", torch_dtype=torch_dtype).to(device)
MODEL.eval()
torch.set_grad_enabled(False)


# =========================
# Utilitas parsing & masking (Tidak Berubah)
# =========================

def split_url_tokens(url: str):
    """Pecah URL menjadi token sederhana dari path & query."""
    parsed = urlparse(url)
    path = unquote(parsed.path or "")
    query = unquote(parsed.query or "")
    delimiters = r"[\/\-\_\=\&\?\.\+\(\)\[\]\<\>\{\}]"
    tokens = re.split(delimiters, path.strip("/")) + re.split(delimiters, query)
    return [tok for tok in tokens if tok]


def iter_urls_from_decoded_csv(csv_path: str, mask_numbers: bool = True):
    """Streaming baca kolom 'url' dari CSV hasil decoder (hemat RAM)."""
    with open(csv_path, "r", encoding="utf-8", errors="ignore", newline="") as f:
        reader = csv.DictReader(f)
        if "url" not in reader.fieldnames:
            raise ValueError(
                f"CSV {csv_path} tidak memiliki kolom 'url'. "
                "Pastikan file dibuat oleh app.decoder dengan flag --csv."
            )
        for row in reader:
            url = row["url"]
            if mask_numbers:
                url = re.sub(r"\d+", "<NUM>", url)
            yield url


def topk_urls(csv_path: str, k: int = 200_000):
    """Hitung frekuensi URL (masked) secara streaming, dan ambil Top-K paling sering."""
    cnt = Counter()
    for url in iter_urls_from_decoded_csv(csv_path):
        cnt[url] += 1
    return [u for u, _ in cnt.most_common(k)]


# =========================
# Embedding (memmap, per-batch) (Tidak Berubah)
# =========================

def generate_url_embeddings_to_memmap(
    url_list,
    memmap_path: str,
    batch_size: int = 128,
    max_length: int = 64,
    verbose: bool = False,
):
    """Tulis embedding BERT ke file memmap (float32) per-batch — hemat RAM."""
    n = len(url_list)
    if n == 0:
        raise ValueError("url_list kosong — tidak ada URL untuk di-embed.")

    out_dir = os.path.dirname(memmap_path)
    if out_dir:
        os.makedirs(out_dir, exist_ok=True)

    hidden_size = MODEL.config.hidden_size
    embs = np.memmap(memmap_path, dtype="float32", mode="w+", shape=(n, hidden_size))
    
    start_time = time.time()
    i = 0
    batch_count = 0
    total_batches = (n + batch_size - 1) // batch_size
    
    while i < n:
        j = min(i + batch_size, n)
        batch = url_list[i:j]
        
        batch_count += 1
        
        inputs = TOKENIZER(
            batch,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=max_length,
        ).to(device)

        with torch.no_grad():
            outputs = MODEL(**inputs)
            batch_emb = outputs.last_hidden_state.mean(dim=1)
            if batch_emb.dtype != torch.float32:
                batch_emb = batch_emb.float()

        embs[i:j, :] = batch_emb.detach().cpu().numpy()
        i = j

        # Progress reporting
        if verbose and batch_count % 10 == 0 or i == n:
            elapsed = time.time() - start_time
            rate = i / elapsed if elapsed > 0 else 0
            
            remaining_seconds = (n - i) / rate if rate > 0 else 0
            
            if remaining_seconds >= 3600:
                eta_str = f"{remaining_seconds/3600:.1f}h"
            elif remaining_seconds >= 60:
                eta_str = f"{remaining_seconds/60:.1f}m"
            else:
                eta_str = f"{remaining_seconds:.1f}s"
                
            print(
                f"[BATCH {batch_count}/{total_batches}] Embedded: {i}/{n} | Rate: {rate:.1f} url/s | ETA: {eta_str}",
                end='\r' if i < n else '\n',
                flush=True
            )

    embs.flush() 
    del embs
    return memmap_path


# =========================
# Evaluasi Metriks (Memastikan Metrik yang Dipilih Digunakan)
# =========================

def evaluate_clustering_metrics(embs: np.ndarray, labels: np.ndarray, metric: str = "cosine"):
    """Hitung metrik evaluasi klastering (Silhouette dan DBI)."""
    # Filter hanya data yang terklaster (label != -1)
    clustered_indices = labels != -1
    X_clustered = embs[clustered_indices]
    labels_clustered = labels[clustered_indices]

    n_clusters_total = len(np.unique(labels)) 
    n_outliers = np.sum(labels == -1)
    
    unique_clustered_labels = np.unique(labels_clustered)
    n_clusters_actual = len(unique_clustered_labels)

    if n_clusters_actual < 2:
        print(f"⚠️ Hanya {n_clusters_actual} klaster valid ditemukan (di luar outlier).")
        print("⚠️ Silhouette dan DBI memerlukan setidaknya 2 klaster.")
        return {
            "N_URLS": len(labels),
            "N_CLUSTERS_TOTAL": n_clusters_total,
            "N_CLUSTERS_ACTUAL": n_clusters_actual,
            "N_OUTLIERS": int(n_outliers),
            "SILHOUETTE_SCORE": np.nan,
            "DAVIES_BOULDIN_INDEX": np.nan,
            "DISTANCE_METRIC": metric,
        }

    print(f"📊 Menghitung Silhouette Score ({metric})...")
    try:
        # Silhouette score menggunakan metrik yang dipilih oleh pengguna
        sil_score = silhouette_score(X_clustered, labels_clustered, metric=metric)
    except Exception as e:
        print(f"❌ Error saat menghitung Silhouette Score: {e}")
        sil_score = np.nan

    print(f"📊 Menghitung Davies-Bouldin Index (Euclidean)...")
    try:
        # Davies-Bouldin Index tidak memiliki parameter 'metric' (hanya Euclidean)
        dbi_score = davies_bouldin_score(X_clustered, labels_clustered)
    except Exception as e:
        print(f"❌ Error saat menghitung Davies-Bouldin Index: {e}")
        dbi_score = np.nan
    
    return {
        "N_URLS": len(labels),
        "N_CLUSTERS_TOTAL": n_clusters_total,
        "N_CLUSTERS_ACTUAL": n_clusters_actual,
        "N_OUTLIERS": int(n_outliers),
        "SILHOUETTE_SCORE": sil_score,
        "DAVIES_BOULDIN_INDEX": dbi_score,
        "DISTANCE_METRIC": metric,
    }

# =========================
# Pipeline utama
# =========================

def main():
    parser = argparse.ArgumentParser(
        description="Cluster NGINX URLs with BERT + OPTICS (RAM-friendly)."
    )
    # ... (Semua Argumen) ...
    parser.add_argument(
        "in_file",
        help="Input CSV hasil decoder (outputs/access_decoded.csv). Gunakan --decoded-csv.",
    )
    parser.add_argument(
        "out_file",
        nargs="?" if "--evaluate" in sys.argv else None, 
        help="Output CSV berisi label cluster per URL (mis. outputs/access_optics.csv). Diabaikan jika --evaluate.",
    )
    parser.add_argument("-m", type=int, default=5, help="OPTICS min_samples")
    parser.add_argument("-e", type=float, default=0.5, help="OPTICS eps")
    parser.add_argument(
        "--topk",
        type=int,
        default=200_000,
        help="Ambil Top-K URL paling sering (semakin kecil semakin hemat RAM).",
    )
    parser.add_argument(
        "--batch-size", type=int, default=128, help="Batch size embedding BERT"
    )
    parser.add_argument(
        "--max-length",
        type=int,
        default=64,
        help="Maks panjang token untuk BERT (64 cukup untuk mayoritas URL).",
    )
    parser.add_argument(
        "--memmap",
        default="outputs/embeddings.dat",
        help="Path file memmap untuk embeddings (akan dibuat).",
    )
    parser.add_argument(
        "--decoded-csv",
        action="store_true",
        help="Wajib diaktifkan: menandakan input adalah CSV hasil decoder.",
    )

    parser.add_argument(
        "--evaluate",
        action="store_true",
        help="Aktifkan mode evaluasi metriks klastering (Silhouette dan DBI). Mengabaikan out_file untuk clustering.",
    )
    parser.add_argument(
        "--metric",
        type=str,
        default="euclidean",
        choices=["euclidean", "cosine"],
        help="Metrik jarak yang digunakan untuk OPTICS dan evaluasi Silhouette Score.",
    )
    
    parser.add_argument(
        "-v",
        "--verbose",
        action="store_true",
        help="Tampilkan rincian progress embedding, termasuk ETA dan rate.",
    )

    # N_jobs untuk Scikit-learn (diabaikan oleh CuML)
    parser.add_argument( 
        "-j",
        "--n-jobs",
        type=int,
        default=-1 if not HAS_GPU_CLUSTERING else 1, 
        help="Jumlah core CPU yang digunakan untuk OPTICS (-1 untuk semua core). Diabaikan oleh CuML.",
    ) 


    args = parser.parse_args()
    
    # --- VALIDASI ARGUMEN ---
    if not args.evaluate and not args.out_file:
        parser.error("argumen 'out_file' wajib diberikan jika --evaluate tidak aktif.")
        
    if not os.path.exists(args.in_file):
        print(f"❌ File tidak ditemukan: '{args.in_file}'")
        sys.exit(1)
    if args.out_file:
        out_dir = os.path.dirname(args.out_file)
        if out_dir and not os.path.exists(out_dir):
            os.makedirs(out_dir, exist_ok=True)

    if not args.decoded_csv:
        print(
            "❌ Harap gunakan --decoded-csv dan berikan input CSV dari app.decoder."
        )
        sys.exit(1)
        
    # LOGIKA BARU: Print mode evaluasi dan metrik yang digunakan
    if args.evaluate:
        print("\n" + "="*50)
        print("         🌟 MODE EVALUASI METRIK AKTIF 🌟")
        print(f"   Metrik Jarak yang digunakan: {args.metric.upper()}")
        print("   (Digunakan untuk OPTICS Clustering dan Silhouette Score)")
        print("="*50)


    # 1) Top-K URL (streaming)
    print("🔎 Menghitung Top-K URL dari decoded CSV (streaming)...")
    unique_urls = topk_urls(args.in_file, k=args.topk)
    print(f"✅ Siap embed: {len(unique_urls):,} URL")

    if len(unique_urls) == 0:
        print("❌ Tidak ada URL yang ditemukan. Periksa input CSV.")
        sys.exit(1)

    # 2) Tokenisasi ringan string URL 
    print("🔤 Menyusun token string URL (ringan)...")
    tokenized_urls = [" ".join(split_url_tokens(u)) for u in unique_urls]

    # 3) Embedding BERT → memmap (hemat RAM)
    print("🧠 Menghitung embedding BERT (stream ke memmap)...")
    memmap_path = generate_url_embeddings_to_memmap(
        tokenized_urls,
        args.memmap,
        batch_size=args.batch_size,
        max_length=args.max_length,
        verbose=args.verbose, 
    )

    # 4) Load memmap readonly, normalisasi, kemudian OPTICS
    print("✨ Normalisasi vektor...")
    hidden_size = MODEL.config.hidden_size
    embs = np.memmap(
        memmap_path,
        dtype="float32",
        mode="r",
        shape=(len(tokenized_urls), hidden_size),
    )
    
    embs_normalized = normalize(embs) # NumPy Array (CPU)
    del embs 

    # --- PENGELOLAAN DATA DAN KLATERING UNTUK GPU/CPU ---
    
    embs_device = embs_normalized # Default: menggunakan NumPy di CPU
    if HAS_GPU_CLUSTERING:
        print("🚀 Transfer data ke GPU (CuPy)...")
        # Konversi NumPy (CPU) ke CuPy (GPU).
        embs_device = cp.asarray(embs_normalized)
        # embs_normalized tetap di CPU untuk evaluasi/penyimpanan
    
    print(f"🧩 OPTICS clustering (min_samples={args.m}, eps={args.e}, metric='{args.metric}')...")
    
    clustering_start_time = time.time()
    
    # Parameter OPTICS
    optics_params = {
        'min_samples': args.m,
        'eps': args.e,
        'metric': args.metric, # Menggunakan metrik yang dipilih
    }
    
    if not HAS_GPU_CLUSTERING: # Tambahkan parameter spesifik scikit-learn jika CPU
        optics_params['n_jobs'] = args.n_jobs
        # optics_params['algorithm'] = 'ball_tree' # Memaksa Ball Tree untuk Cosine/Euclidean dengan n_jobs
    
    # Lakukan Klastering (hanya perlu fit_predict satu kali)
    optics = OPTICS_IMPL(**optics_params)
    labels_device = optics.fit_predict(embs_device)
    
    clustering_elapsed = time.time() - clustering_start_time

    # --- KONVERSI HASIL KEMBALI KE CPU ---
    if HAS_GPU_CLUSTERING:
        # Pindahkan label hasil CuPy (GPU) ke NumPy (CPU)
        labels = cp.asnumpy(labels_device)
        del embs_device # Bebaskan memori GPU
    else:
        labels = labels_device # Jika CPU, labels sudah berupa NumPy array
        
    print(f"✅ OPTICS selesai dalam {clustering_elapsed:.2f} detik.")
    # --- AKHIR PENGELOLAAN DATA ---


    # 5) Evaluasi Metrik (Jika diminta)
    if args.evaluate:
        
        # Panggil evaluate_clustering_metrics dengan args.metric yang dipilih
        print("\n" + "="*50)
        print("         📊 EVALUASI METRIK 📉")
        print("="*50)
        
        metrics = evaluate_clustering_metrics(embs_normalized, labels, args.metric)
        
        # ... (Cetak hasil metrics) ...
        print("\n" + "="*50)
        print("          📈 HASIL METRIK KLATERING 📉")
        print("="*50)
        print(f"Total URL diproses: {metrics['N_URLS']:,}")
        print(f"Waktu Klastering OPTICS: {clustering_elapsed:.2f} detik")
        print(f"Metrik Jarak OPTICS/Silhouette: {metrics['DISTANCE_METRIC']}") 
        print(f"OPTICS min_samples: {args.m}, eps: {args.e}")
        print("-" * 50)
        print(f"Jumlah Klaster Nyata (selain -1): {metrics['N_CLUSTERS_ACTUAL']}")
        print(f"Jumlah Outlier (Label -1): {metrics['N_OUTLIERS']:,}")
        print("-" * 50)
        
        if not np.isnan(metrics['SILHOUETTE_SCORE']):
            print(f"Silhouette Score (Ideal: mendekati 1.0): {metrics['SILHOUETTE_SCORE']:.4f}")
        else:
            print(f"Silhouette Score: {metrics['SILHOUETTE_SCORE']}")
            
        if not np.isnan(metrics['DAVIES_BOULDIN_INDEX']):
            print(f"Davies-Bouldin Index (Ideal: mendekati 0.0, Euclidean): {metrics['DAVIES_BOULDIN_INDEX']:.4f}") 
        else:
            print(f"Davies-Bouldin Index: {metrics['DAVIES_BOULDIN_INDEX']} (Euclidean)")
            
        print("="*50)
        
        del embs_normalized
        sys.exit(0) 

    # 6) Simpan hasil 
    del embs_normalized 
    
    print("💾 Menyimpan hasil label...")
    df_label = pd.DataFrame({"masked": unique_urls, "cluster": labels}).sort_values(
        by="cluster"
    )
    df_label.to_csv(args.out_file, index=False, encoding="utf-8")
    print(f"✅ Clustering results saved to: {args.out_file}")

    # 7) Simpan juga daftar cluster → .txt (opsional)
    txt_path = f"{args.out_file}.txt"
    unique_labels = sorted(set(labels), key=lambda x: (x == -1, x))
    groups = {lab: [] for lab in unique_labels}
    for i, lab in enumerate(labels):
        groups[lab].append(unique_urls[i])
    with open(txt_path, "w", encoding="utf-8") as f:
        for lab in unique_labels:
            title = "Noise (-1)" if lab == -1 else f"Cluster {lab}"
            f.write(f"\n{title} ({len(groups[lab])} items):\n")
            for u in groups[lab]:
                f.write(f"  {u}\n")
    print(f"📝 Cluster listing disimpan: {txt_path}")


if __name__ == "__main__":
    main()

Overwriting app/main.py


In [8]:
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [9]:
!ls /content/drive/MyDrive/datasets

archive_2.zip  web_logs


In [10]:
# !unzip /content/drive/MyDrive/datasets/archive_2.zip -d /content/drive/MyDrive/datasets/web_logs

In [11]:
!cp -r /content/drive/MyDrive/datasets/web_logs/access.log /content/inputs


In [12]:
!ls /content/inputs


access.log  sample.log


In [14]:
!python -c "print(sum(1 for _ in open('inputs/access.log', encoding='utf-8', errors='ignore')))"


10365152


In [6]:
!python -c "print(sum(1 for _ in open('outputs/access_decoded.csv', encoding='utf-8', errors='ignore')))"


9283833


In [1]:
!.\.venv\Scripts\python -m app.decoder inputs/sample.log outputs/sample_decoded.csv --csv


⏳ Starting streaming decode to CSV...
✅ Streaming decode complete.
✅ Log file successfully converted to outputs/sample_decoded.csv


In [ ]:
!.\.venv\Scripts\python -u -m app.main outputs/access_decoded.csv outputs/access_optics2.csv -m 10 --topk 200000 --batch-size 128 --max-length 64 --decoded-csv --metric euclidean -v

UsageError: Cell magic `%%powershell` not found.


In [2]:
# import numpy as np
# import pandas as pd
# from sklearn.metrics import silhouette_score, davies_bouldin_score

# # Baca hasil clustering
# df = pd.read_csv("outputs/access_optics.csv")
# labels = df["cluster"].values

# # Baca embeddings yang sudah disimpan
# X = np.load("outputs/embeddings.npy")

# # Jumlah cluster & outlier
# n_outliers = np.sum(labels == -1)
# n_clusters = len(set(labels)) - (1 if -1 in labels else 0)

# print(f"Jumlah cluster terbentuk: {n_clusters}")
# print(f"Jumlah outlier terdeteksi: {n_outliers}")

# # Evaluasi kualitas cluster
# mask = labels != -1
# if np.sum(mask) > 1 and n_clusters > 1:
#     sil_score = silhouette_score(X[mask], labels[mask])
#     dbi_score = davies_bouldin_score(X[mask], labels[mask])
#     print(f"Silhouette Score: {sil_score:.4f}")
#     print(f"Davies-Bouldin Index: {dbi_score:.4f}")
# else:
#     print("⚠️ Tidak cukup cluster untuk menghitung Silhouette/DBI")


In [ ]:
# %%writefile app/report.py
# # app/report.py
# import os
# import argparse
# import pandas as pd
# from collections import defaultdict
# from html import escape

# def generate_html_report(csv_path: str, html_path: str, title: str = "OPTICS URL Clusters",
#                          show_noise=True, max_items_per_cluster=None):
#     """
#     Membuat laporan HTML dari file CSV hasil clustering.
#     CSV diharapkan punya kolom: 'masked' dan 'cluster'.
#     """
#     if not os.path.exists(csv_path):
#         raise FileNotFoundError(f"CSV not found: {csv_path}")

#     df = pd.read_csv(csv_path)
#     if not {"masked", "cluster"}.issubset(df.columns):
#         raise ValueError("CSV must contain columns: 'masked' and 'cluster'")

#     # pastikan tipe cluster numerik (agar -1 terdeteksi sebagai noise)
#     try:
#         df["cluster"] = pd.to_numeric(df["cluster"])
#     except Exception:
#         pass

#     total_items = len(df)
#     noise_mask = df["cluster"].eq(-1) if "cluster" in df else pd.Series([False]*len(df))
#     noise_count = int(noise_mask.sum())
#     if not show_noise:
#         df = df[~noise_mask]

#     # kelompokkan
#     grouped = defaultdict(list)
#     for _, row in df.iterrows():
#         grouped[row["cluster"]].append(str(row["masked"]))

#     # urutkan cluster by size desc (kecuali -1 selalu taruh paling akhir)
#     def sort_key(k):
#         if k == -1:
#             return (1, 10**12)  # noise di akhir
#         return (0, -len(grouped[k]))

#     clusters_sorted = sorted(grouped.keys(), key=sort_key)
#     num_clusters_non_noise = sum(1 for cid in clusters_sorted if cid != -1)

#     # HTML template (inline CSS + sedikit JS)
#     css = """
#     body { font-family: system-ui, -apple-system, Segoe UI, Roboto, Arial, sans-serif; margin: 24px; }
#     h1 { margin-bottom: 0.2rem; }
#     .muted { color: #666; }
#     .stats { display: flex; gap: 16px; flex-wrap: wrap; margin: 12px 0 20px; }
#     .stat { background: #f4f6f8; border: 1px solid #e5e7eb; border-radius: 12px; padding: 12px 16px; }
#     .search { margin: 16px 0 22px; }
#     input[type="search"] { width: 100%; padding: 10px 12px; border: 1px solid #d1d5db; border-radius: 10px; }
#     details { border: 1px solid #e5e7eb; border-radius: 12px; margin: 10px 0; overflow: hidden; background: #fff; }
#     summary { cursor: pointer; padding: 12px 16px; background: #f9fafb; font-weight: 600; display: flex; justify-content: space-between; align-items: center; }
#     .count { font-size: 12px; background: #eef2ff; color: #3730a3; border-radius: 999px; padding: 4px 8px; }
#     ul { margin: 0; padding: 10px 22px 16px 32px; }
#     li { line-height: 1.5; word-break: break-all; }
#     .noise { background: #fff7ed; }
#     .hidden { display: none !important; }
#     footer { margin-top: 28px; font-size: 12px; color: #888; }
#     """

#     js = """
#     function filterClusters() {
#       const q = document.getElementById('q').value.toLowerCase();
#       const blocks = document.querySelectorAll('.cluster');
#       blocks.forEach(block => {
#         const items = Array.from(block.querySelectorAll('li'));
#         let any = false;
#         items.forEach(li => {
#           const show = li.textContent.toLowerCase().includes(q);
#           li.classList.toggle('hidden', !show);
#           if (show) any = true;
#         });
#         block.classList.toggle('hidden', !any);
#       });
#       // update summary counts after filter
#       document.querySelectorAll('.cluster').forEach(block => {
#         const total = block.querySelectorAll('li:not(.hidden)').length;
#         const badge = block.querySelector('.count');
#         if (badge) badge.textContent = total + " item";
#       });
#     }
#     """

#     # build body
#     parts = []
#     parts.append(f"<h1>{escape(title)}</h1>")
#     parts.append(f'<div class="muted">Sumber: {escape(os.path.basename(csv_path))}</div>')
#     parts.append(
#         '<div class="stats">'
#         f'<div class="stat"><b>Total item</b><br>{total_items:,}</div>'
#         f'<div class="stat"><b>Total cluster</b><br>{num_clusters_non_noise:,}</div>'
#         f'<div class="stat"><b>Noise (-1)</b><br>{noise_count:,}</div>'
#         '</div>'
#     )
#     parts.append(
#         '<div class="search">'
#         '<input id="q" type="search" placeholder="Cari URL di semua cluster…" oninput="filterClusters()">'
#         '</div>'
#     )

#     for cid in clusters_sorted:
#         items = grouped[cid]
#         if max_items_per_cluster is not None:
#             shown = items[:max_items_per_cluster]
#             truncated = len(items) - len(shown)
#         else:
#             shown = items
#             truncated = 0

#         is_noise = (cid == -1)
#         cls = "cluster noise" if is_noise else "cluster"
#         label = "Noise (-1)" if is_noise else f"Cluster {cid}"
#         count_str = f"{len(shown):,} item" if truncated == 0 else f"{len(shown):,} item (+"+f"{truncated:,} tersembunyi)"
#         parts.append(f'<details class="{cls}"><summary>{escape(label)} <span class="count">{count_str}</span></summary>')
#         parts.append("<ul>")
#         for url in shown:
#             parts.append(f"<li>{escape(url)}</li>")
#         if truncated > 0:
#             parts.append(f'<li class="muted">… dan {truncated:,} item lainnya tidak ditampilkan</li>')
#         parts.append("</ul></details>")

#     parts.append('<footer>Generated by report.py</footer>')

#     html = f"""<!doctype html>
# <html lang="id">
# <head>
# <meta charset="utf-8">
# <meta name="viewport" content="width=device-width,initial-scale=1">
# <title>{escape(title)}</title>
# <style>{css}</style>
# </head>
# <body>
# {''.join(parts)}
# <script>{js}</script>
# </body>
# </html>"""

#     os.makedirs(os.path.dirname(html_path) or ".", exist_ok=True)
#     with open(html_path, "w", encoding="utf-8") as f:
#         f.write(html)
#     return html_path

# def main():
#     p = argparse.ArgumentParser(description="Generate HTML report from clustering CSV")
#     p.add_argument("csv", help="Path ke outputs/access_optics.csv")
#     p.add_argument("html", help="Path output .html (mis. outputs/cluster_report.html)")
#     p.add_argument("--title", default="OPTICS URL Clusters", help="Judul laporan")
#     p.add_argument("--hide-noise", action="store_true", help="Sembunyikan cluster -1 (noise)")
#     p.add_argument("--max-per-cluster", type=int, default=None, help="Batasi jumlah item ditampilkan per cluster")
#     args = p.parse_args()

#     path = generate_html_report(
#         csv_path=args.csv,
#         html_path=args.html,
#         title=args.title,
#         show_noise=not args.hide_noise,
#         max_items_per_cluster=args.max_per_cluster
#     )
#     print(f"✅ Report tersimpan: {path}")

# if __name__ == "__main__":
#     main()


Writing app/report.py


In [ ]:
!.\.venv\Scripts\python -m app.report outputs/access_optics.csv outputs/cluster_report.html

✅ Report tersimpan: outputs/cluster_report.html


In [21]:
%%writefile app/expand_originals.py
import os
import re
import csv
import argparse
import sqlite3
from collections import defaultdict, Counter
from html import escape
import math

MASK_RE = re.compile(r"\d+")

def mask_url(url: str) -> str:
    return MASK_RE.sub("<NUM>", url or "")

def load_clustered_masked(masked_cluster_csv):
    masked_to_cluster = {}
    cluster_to_masked = defaultdict(set)
    with open(masked_cluster_csv, "r", encoding="utf-8", newline="") as f:
        r = csv.DictReader(f)
        if "masked" not in r.fieldnames or "cluster" not in r.fieldnames:
            raise ValueError("CSV cluster harus punya kolom 'masked' dan 'cluster'")
        for row in r:
            m = row["masked"]
            c = int(row["cluster"])
            masked_to_cluster[m] = c
            cluster_to_masked[c].add(m)
    return masked_to_cluster, cluster_to_masked

# -------- Aggregator per original URL → per-IP buckets --------
class OrigAgg:
    __slots__ = ("total", "ip_cnt", "ip_sample")
    def __init__(self):
        self.total = 0
        self.ip_cnt = Counter()
        self.ip_sample = {}  # ip -> first-seen row dict

def format_log_line(row: dict) -> str:
    def g(k, default=""):
        v = row.get(k, default) if row else default
        return "" if v is None else str(v)
    ip = g("ip", "-")
    time = g("time", "-")
    method = g("method", "-")
    url = g("url", "-")
    protocol = g("protocol", "-")
    status = g("status", "-")
    size = g("size", "-")
    ref = g("referrer", "-")
    ua = g("user_agent", "-")
    extra = g("extra", "-")
    return f'{ip} - - [{time}] "{method} {url} {protocol}" {status} {size} "{ref}" "{ua}" "{extra}"'

def stream_count_originals(decoded_csv, masked_to_cluster,
                            max_originals_per_masked=200,
                            top_ips_per_original=3):
    masked_counts = Counter()
    store: dict[str, dict[str, OrigAgg]] = defaultdict(dict)

    with open(decoded_csv, "r", encoding="utf-8", errors="ignore", newline="") as f:
        r = csv.DictReader(f)
        need = {"ip","time","method","url","protocol","status","size","referrer","user_agent","extra"}
        if not need.issubset(set(r.fieldnames or [])):
            raise ValueError("CSV decoded harus punya kolom: " + ", ".join(sorted(need)))

        for row in r:
            orig = row["url"]
            m = mask_url(orig)
            if m not in masked_to_cluster:
                continue

            masked_counts[m] += 1
            bucket = store[m]

            if orig not in bucket and len(bucket) >= max_originals_per_masked:
                top = sorted(bucket.items(), key=lambda kv: kv[1].total, reverse=True)[:max_originals_per_masked]
                bucket.clear()
                bucket.update(top)

            agg = bucket.get(orig)
            if agg is None:
                agg = OrigAgg()
                bucket[orig] = agg

            agg.total += 1
            ip = row.get("ip") or "-"
            agg.ip_cnt[ip] += 1
            if ip not in agg.ip_sample:  # first-seen per IP
                agg.ip_sample[ip] = {
                    "ip": row.get("ip"),
                    "time": row.get("time"),
                    "method": row.get("method"),
                    "url": row.get("url"),
                    "protocol": row.get("protocol"),
                    "status": row.get("status"),
                    "size": row.get("size"),
                    "referrer": row.get("referrer"),
                    "user_agent": row.get("user_agent"),
                    "extra": row.get("extra"),
                }

    # optional trim per original
    if top_ips_per_original is not None:
        for _, bucket in store.items():
            for _, agg in bucket.items():
                keep_ips = set(ip for ip, _ in agg.ip_cnt.most_common(top_ips_per_original*4))
                agg.ip_cnt = Counter({ip: cnt for ip, cnt in agg.ip_cnt.items() if ip in keep_ips})
                agg.ip_sample = {ip: agg.ip_sample[ip] for ip in list(agg.ip_sample.keys()) if ip in keep_ips}

    return masked_counts, store

# ---------- SQLite ----------
def init_db(db_path):
    os.makedirs(os.path.dirname(db_path) or ".", exist_ok=True)
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()
    cur.execute("PRAGMA journal_mode=WAL;")
    cur.execute("PRAGMA synchronous=NORMAL;")
    cur.executescript("""
    DROP TABLE IF EXISTS clusters;
    DROP TABLE IF EXISTS masked;
    DROP TABLE IF EXISTS originals;
    DROP TABLE IF EXISTS original_ips;

    CREATE TABLE clusters(
      cluster_id INTEGER PRIMARY KEY
    );

    CREATE TABLE masked(
      masked TEXT PRIMARY KEY,
      cluster_id INTEGER,
      masked_total INTEGER,
      FOREIGN KEY(cluster_id) REFERENCES clusters(cluster_id)
    );

    CREATE TABLE originals(
      masked TEXT,
      original_url TEXT,
      total_cnt INTEGER,
      PRIMARY KEY(masked, original_url),
      FOREIGN KEY(masked) REFERENCES masked(masked)
    );

    CREATE TABLE original_ips(
      masked TEXT,
      original_url TEXT,
      ip TEXT,
      cnt INTEGER,
      sample_line TEXT,
      PRIMARY KEY(masked, original_url, ip),
      FOREIGN KEY(masked, original_url) REFERENCES originals(masked, original_url)
    );

    CREATE INDEX idx_masked_cluster ON masked(cluster_id);
    CREATE INDEX idx_originals_masked ON originals(masked);
    CREATE INDEX idx_origips_masked ON original_ips(masked);
    """)
    conn.commit()
    return conn

def save_to_sqlite(db_path, masked_to_cluster, masked_counts, store,
                   top_per_masked=20, top_ips_per_original=3):
    conn = init_db(db_path)
    cur = conn.cursor()

    clusters = sorted(set(masked_to_cluster.values()), key=lambda x:(x==-1, x))
    cur.executemany("INSERT INTO clusters(cluster_id) VALUES(?)", [(c,) for c in clusters])

    cur.executemany(
        "INSERT INTO masked(masked, cluster_id, masked_total) VALUES(?,?,?)",
        [(m, masked_to_cluster[m], int(masked_counts.get(m, 0))) for m in masked_to_cluster]
    )

    batch_orig, batch_ip = [], []
    for m, bucket in store.items():
        for orig, agg in sorted(bucket.items(), key=lambda kv: kv[1].total, reverse=True)[:top_per_masked]:
            batch_orig.append((m, orig, int(agg.total)))
            for ip, cnt in agg.ip_cnt.most_common(top_ips_per_original):
                sample = format_log_line(agg.ip_sample.get(ip))
                batch_ip.append((m, orig, ip, int(cnt), sample))
            if len(batch_orig) >= 50_000:
                cur.executemany("INSERT OR REPLACE INTO originals(masked, original_url, total_cnt) VALUES(?,?,?)", batch_orig)
                batch_orig.clear()
            if len(batch_ip) >= 50_000:
                cur.executemany("INSERT OR REPLACE INTO original_ips(masked, original_url, ip, cnt, sample_line) VALUES(?,?,?,?,?)", batch_ip)
                batch_ip.clear()

    if batch_orig:
        cur.executemany("INSERT OR REPLACE INTO originals(masked, original_url, total_cnt) VALUES(?,?,?)", batch_orig)
    if batch_ip:
        cur.executemany("INSERT OR REPLACE INTO original_ips(masked, original_url, ip, cnt, sample_line) VALUES(?,?,?,?,?)", batch_ip)

    conn.commit()
    conn.close()

def _q(conn, sql, args=()):
    cur = conn.execute(sql, args)
    cols = [c[0] for c in cur.description]
    for row in cur:
        yield dict(zip(cols, row))

BASE_CSS = """
body{font-family:Arial, sans-serif; margin:24px;}
details{border:1px solid #ddd; border-radius:10px; padding:10px 12px; margin:12px 0; background:#fff;}
summary{cursor:pointer; font-weight:bold; outline:none}
.badge{display:inline-block; padding:2px 8px; border-radius:12px; background:#eef; margin-left:8px; color:#334}
table{border-collapse:collapse; width:100%; margin:8px 0 14px}
th,td{border:1px solid #eee; padding:6px; text-align:left}
th{background:#fafafa}
.small{color:#666; font-size:12px}
pre{margin:0; font-family:ui-monospace, SFMono-Regular, Menlo, Consolas, "Liberation Mono", monospace; font-size:12px; white-space:pre-wrap}
"""

def write_text(path, text):
    os.makedirs(os.path.dirname(path) or ".", exist_ok=True)
    with open(path, "w", encoding="utf-8") as f:
        f.write(text)

# ---------- Single-page COLLAPSIBLE (tanpa search & sorting) ----------
def generate_collapsible_single_html(
    db_path,
    out_html,
    top_per_masked=20,
    max_masked_per_cluster=200,
    top_ips_per_original=3,
):
    """
    Single-page HTML:
      - Cluster → <details>
      - Di dalam cluster: render hingga N masked (MUNCUL SEKALI per masked)
      - Untuk tiap masked:
          * tampilkan ringkasan (masked, total hits)
          * tabel Original URL (top-K)
          * untuk setiap original URL: tabel IP (top-M IP) dengan sample-line per IP
    """
    conn = sqlite3.connect(db_path)
    clusters = list(_q(conn, """
        SELECT cluster_id, COUNT(*) AS n_masked, SUM(masked_total) AS hits
        FROM masked GROUP BY cluster_id ORDER BY (cluster_id=-1), cluster_id
    """))

    blocks = []
    total_masked_rendered = 0

    for c in clusters:
        cid = c["cluster_id"]
        title = "Noise (-1)" if cid == -1 else f"Cluster {cid}"
        n_masked = c["n_masked"]
        hits = c["hits"] or 0

        # ambil masked yang terbesar di cluster ini
        masked_rows = list(_q(conn, """
            SELECT masked, masked_total
            FROM masked
            WHERE cluster_id=?
            ORDER BY masked_total DESC, masked
            LIMIT ?
        """, (cid, max_masked_per_cluster)))

        masked_blocks = []
        for row in masked_rows:
            m = row["masked"]
            masked_total = row["masked_total"] or 0

            # top-N original URL untuk masked ini
            originals = list(_q(conn, """
                SELECT original_url, total_cnt
                FROM originals
                WHERE masked=?
                ORDER BY total_cnt DESC
                LIMIT ?
            """, (m, top_per_masked)))

            # bangun tabel original URL
            orig_rows_html = []
            for orig in originals:
                o_url = orig["original_url"]
                o_total = orig["total_cnt"] or 0

                # top-M IP untuk original ini
                ip_rows = list(_q(conn, """
                    SELECT ip, cnt, sample_line
                    FROM original_ips
                    WHERE masked=? AND original_url=?
                    ORDER BY cnt DESC
                    LIMIT ?
                """, (m, o_url, top_ips_per_original)))

                # tabel IP (di-bungkus agar rapi)
                ip_tbl_rows = []
                for ipr in ip_rows:
                    share = (ipr["cnt"] / masked_total) if masked_total else 0.0
                    ip_tbl_rows.append(
                        "<tr>"
                        f"<td>{escape(ipr['ip'] or '-')}</td>"
                        f"<td><pre>{escape(ipr['sample_line'] or '')}</pre></td>"
                        f"<td>{ipr['cnt']}</td>"
                        f"<td>{share:.2%}</td>"
                        "</tr>"
                    )

                ip_table_html = (
                    "<table>"
                    "<thead><tr><th>IP</th><th>Sample Log Line (first seen per IP)</th><th>Count</th><th>Share</th></tr></thead>"
                    f"<tbody>{''.join(ip_tbl_rows)}</tbody>"
                    "</table>"
                )

                # baris untuk original URL (URL dan totalnya), lalu tabel IP di bawahnya
                orig_rows_html.append(
                    "<tr>"
                    f"<td style='width:40%;word-break:break-all'>{escape(o_url)}</td>"
                    f"<td style='width:10%'>{o_total}</td>"
                    f"<td>{ip_table_html}</td>"
                    "</tr>"
                )

            if not orig_rows_html:
                # kalau tidak ada original (jarang terjadi), skip blok masked
                continue

            # satu blok <details> PER MASKED (muncul sekali)
            masked_blocks.append(f"""
            <details>
              <summary><code>{escape(m)}</code>
                <span class="badge">masked hits: {masked_total}</span>
                <span class="badge">showing top {top_per_masked} originals × top {top_ips_per_original} IP</span>
              </summary>
              <table>
                <thead>
                  <tr><th>Original URL</th><th>Total Hits (URL)</th><th>Per-IP Samples</th></tr>
                </thead>
                <tbody>
                  {''.join(orig_rows_html)}
                </tbody>
              </table>
            </details>
            """)

        cluster_html = f"""
        <div class="cluster-container">
          <details>
            <summary>{escape(title)}
              <span class="badge">masked: {n_masked}</span>
              <span class="badge">hits: {hits}</span>
              <span class="small">render up to {min(max_masked_per_cluster, n_masked)} masked</span>
            </summary>
            {''.join(masked_blocks)}
          </details>
        </div>
        """
        blocks.append(cluster_html)
        total_masked_rendered += len(masked_rows)

    conn.close()

    html = f"""<!doctype html>
<html><head><meta charset="utf-8">
<title>Cluster Report (Cluster → Masked → Original → IP)</title>
<style>
{BASE_CSS}
/* kecilkan pre agar muat */
pre{{margin:0; font-size:12px; white-space:pre-wrap}}
code{{background:#f6f8fa; padding:2px 6px; border-radius:6px}}
summary .badge{{margin-left:6px}}
table td{{vertical-align:top}}
</style>
</head><body>
<h2>Cluster Report — Grouped by Masked (no search/sort)</h2>
<div class="small">Rendered masked total (max per cluster): {total_masked_rendered}</div>
{"".join(blocks)}
</body></html>"""
    write_text(out_html, html)


# ---------- (opsional) Multi-page sederhana (tanpa search/sort) ----------
def generate_site_from_sqlite(db_path, outdir, page_size=50, top_per_masked=20, top_ips_per_original=3):
    conn = sqlite3.connect(db_path)

    clusters = list(_q(conn, "SELECT cluster_id, COUNT(*) AS n_masked, SUM(masked_total) AS hits FROM masked GROUP BY cluster_id ORDER BY (cluster_id=-1), cluster_id"))
    rows = "".join(
        f"<tr><td><a href='cluster_{c['cluster_id']}_1.html'>{'Noise (-1)' if c['cluster_id']==-1 else 'Cluster '+str(c['cluster_id'])}</a></td>"
        f"<td>{c['n_masked']}</td><td>{c['hits'] or 0}</td></tr>"
        for c in clusters
    )
    idx_html = f"""<!doctype html><html><head><meta charset='utf-8'><title>Clusters Index</title>
<style>{BASE_CSS}</style></head><body>
<h2>Clusters Index</h2>
<table><thead><tr><th>Cluster</th><th># Masked</th><th>Total Hits</th></tr></thead><tbody>
{rows}
</tbody></table>
</body></html>"""
    write_text(os.path.join(outdir, "index.html"), idx_html)

    cluster_ids = [c["cluster_id"] for c in clusters]
    for cid in cluster_ids:
        total = next(_q(conn, "SELECT COUNT(*) AS n FROM masked WHERE cluster_id=?", (cid,)))["n"]
        pages = max(1, math.ceil(total / page_size))
        for page in range(1, pages+1):
            offset = (page-1)*page_size
            masked_rows = list(_q(conn,
                "SELECT masked, masked_total FROM masked WHERE cluster_id=? ORDER BY masked_total DESC, masked LIMIT ? OFFSET ?",
                (cid, page_size, offset)
            ))
            title = "Noise (-1)" if cid == -1 else f"Cluster {cid}"

            blocks = []
            for row in masked_rows:
                m = row["masked"]
                masked_total = row["masked_total"] or 0
                originals = list(_q(conn,
                    "SELECT original_url, total_cnt FROM originals WHERE masked=? ORDER BY total_cnt DESC LIMIT ?",
                    (m, top_per_masked)
                ))
                orows = ""
                for orig in originals:
                    ip_rows = list(_q(conn,
                        "SELECT ip, cnt, sample_line FROM original_ips WHERE masked=? AND original_url=? ORDER BY cnt DESC LIMIT ?",
                        (m, orig["original_url"], top_ips_per_original)
                    ))
                    for ipr in ip_rows:
                        share = (ipr["cnt"] / masked_total) if masked_total else 0.0
                        orows += f"<tr><td>{escape(ipr['ip'] or '-')}</td><td><pre>{escape(ipr['sample_line'] or '')}</pre></td><td>{ipr['cnt']}</td><td>{share:.2%}</td></tr>"

                blocks.append(f"""
                <div class='cluster-container'>
                  <div><strong>{escape(m)}</strong>
                    <span class='badge'>hits: {masked_total}</span>
                  </div>
                  <table><thead><tr><th>IP</th><th>Sample Log Line</th><th>Count</th><th>Share</th></tr></thead>
                  <tbody>{orows}</tbody></table>
                </div>
                """)

            pager = " ".join(
                (f"<strong>{p}</strong>" if p==page else f"<a href='cluster_{cid}_{p}.html'>{p}</a>")
                for p in range(1, pages+1)
            )
            html = f"""<!doctype html><html><head><meta charset='utf-8'><title>{title}</title>
<style>{BASE_CSS}</style></head><body>
<p><a href='index.html'>&larr; Back to index</a></p>
<h2>{title}</h2>
<div class='pager'>Pages: {pager}</div>
{''.join(blocks)}
<div class='pager'>Pages: {pager}</div>
</body></html>"""
            os.makedirs(outdir, exist_ok=True)
            write_text(os.path.join(outdir, f"cluster_{cid}_{page}.html"), html)

    conn.close()

def main():
    ap = argparse.ArgumentParser(description="Expand masked clusters → per-original per-IP sample lines (SQLite, no search/sort).")
    ap.add_argument("decoded_csv", help="outputs/access_decoded.csv (hasil decoder)")
    ap.add_argument("cluster_csv", help="outputs/access_optics.csv (masked,cluster)")
    ap.add_argument("--top-per-masked", type=int, default=20, help="Top-N original URL per masked")
    ap.add_argument("--top-ips-per-original", type=int, default=3, help="Top-M IP per original URL")
    ap.add_argument("--max-originals-per-masked", type=int, default=200, help="Batas variasi original per masked")

    # SQLite + HTML
    ap.add_argument("--db", default="outputs/cluster_report.db", help="SQLite DB output")
    ap.add_argument("--single-html", default="outputs/expanded_collapsible.html", help="Single-page HTML (collapsible, no search/sort)")
    ap.add_argument("--max-masked-per-cluster", type=int, default=200, help="Batas masked dirender per cluster")

    # (opsional) Multi-page
    ap.add_argument("--site-dir", default="", help="Folder output HTML statis (multi-page, optional)")
    ap.add_argument("--page-size", type=int, default=50, help="# masked per halaman (multi-page optional)")
    args = ap.parse_args()

    print("🔹 Load clustered masked → cluster...")
    masked_to_cluster, _ = load_clustered_masked(args.cluster_csv)
    print(f"  masked patterns: {len(masked_to_cluster):,}")

    print("🔹 Streaming decoded CSV → agregasi per original & per-IP...")
    masked_counts, store = stream_count_originals(
        args.decoded_csv,
        masked_to_cluster,
        max_originals_per_masked=args.max_originals_per_masked,
        top_ips_per_original=args.top_ips_per_original,
    )

    print(f"🔹 Simpan ke SQLite: {args.db}")
    save_to_sqlite(args.db, masked_to_cluster, masked_counts, store,
                   top_per_masked=args.top_per_masked,
                   top_ips_per_original=args.top_ips_per_original)

    if args.single_html:
        print(f"🔹 Generate single HTML (collapsible) → {args.single_html}")
        generate_collapsible_single_html(args.db, args.single_html,
                                         top_per_masked=args.top_per_masked,
                                         max_masked_per_cluster=args.max_masked_per_cluster,
                                         top_ips_per_original=args.top_ips_per_original)

    if args.site_dir:
        print(f"🔹 Generate HTML statis (multi-page) → {args.site_dir}")
        generate_site_from_sqlite(args.db, args.site_dir,
                                  page_size=args.page_size,
                                  top_per_masked=args.top_per_masked,
                                  top_ips_per_original=args.top_ips_per_original)

    print("✅ Done.")
if __name__ == "__main__":
    main()


Overwriting app/expand_originals.py


In [22]:
! .\.venv\Scripts\python -m app.expand_originals outputs\access_decoded.csv outputs\access_optics.csv --db outputs\cluster_report.db --single-html outputs\expanded_collapsible.html --top-per-masked 20 --top-ips-per-original 3 --max-originals-per-masked 200 --max-masked-per-cluster 200

🔹 Load clustered masked → cluster...
  masked patterns: 92,150
🔹 Streaming decoded CSV → agregasi per original & per-IP...
🔹 Simpan ke SQLite: outputs\cluster_report.db
🔹 Generate single HTML (collapsible) → outputs\expanded_collapsible.html
✅ Done.
